# Imports

In [1]:
import numpy as np
from scipy.fft import *

In [2]:
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

## Setup inputs

In [781]:
N = 8

f_base = np.arange(N, dtype=np.float64)+1
f_grid = np.arange(N**2, dtype=np.float64).reshape(N, N)+1

f_grid

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 9., 10., 11., 12., 13., 14., 15., 16.],
       [17., 18., 19., 20., 21., 22., 23., 24.],
       [25., 26., 27., 28., 29., 30., 31., 32.],
       [33., 34., 35., 36., 37., 38., 39., 40.],
       [41., 42., 43., 44., 45., 46., 47., 48.],
       [49., 50., 51., 52., 53., 54., 55., 56.],
       [57., 58., 59., 60., 61., 62., 63., 64.]])

In [783]:
L = 1
sig = 0.2

h = L/N;
x = np.arange(N, dtype=np.float64)*h;
y = np.arange(N, dtype=np.float64)*h;

X, Y = np.meshgrid(x,y);
rsq = (X-0.5*L)**2 + (Y-0.5*L)**2;
sigsq = sig**2;
f_grid = np.exp(-rsq/(2*sigsq))*(rsq - 2*sigsq)/(sigsq**2);
f_grid

array([[  0.51,   1.47,   2.92,   4.19,   4.67,   4.19,   2.92,   1.47],
       [  1.47,   3.74,   6.07,   6.76,   6.53,   6.76,   6.07,   3.74],
       [  2.92,   6.07,   5.9 ,  -0.44,  -5.01,  -0.44,   5.9 ,   6.07],
       [  4.19,   6.76,  -0.44, -20.62, -33.1 , -20.62,  -0.44,   6.76],
       [  4.67,   6.53,  -5.01, -33.1 , -50.  , -33.1 ,  -5.01,   6.53],
       [  4.19,   6.76,  -0.44, -20.62, -33.1 , -20.62,  -0.44,   6.76],
       [  2.92,   6.07,   5.9 ,  -0.44,  -5.01,  -0.44,   5.9 ,   6.07],
       [  1.47,   3.74,   6.07,   6.76,   6.53,   6.76,   6.07,   3.74]])

# Transforms

## 1. DCTs

### 1.1. DCT Type I

In [772]:
f_base_dct1_1d = dct(f_base, 1, norm="ortho", orthogonalize=True)

f_ext_dct1_1d = np.concatenate((f_base, f_base[-2:0:-1]))

f_ext_dct1_1d[0] *= np.sqrt(2)
f_ext_dct1_1d[N-1] *= np.sqrt(2)

res_dct1_1d = (rfft(f_ext_dct1_1d)).real * np.sqrt(2/(N-1)) * 0.5

res_dct1_1d[0] /= np.sqrt(2)
res_dct1_1d[N-1] /= np.sqrt(2)

In [773]:
print("Error: ", np.sum(np.abs(f_base_dct1_1d-res_dct1_1d)))
f_base_dct1_1d, res_dct1_1d

Error:  8.881784197001252e-16


(array([ 4.93, -2.14,  0.85, -0.65]), array([ 4.93, -2.14,  0.85, -0.65]))

In [779]:
f_base_dct1_2d = dctn(f_grid, 1, norm="ortho", orthogonalize=True)

f_ext_dct1_1d = np.zeros((2*N-2, 2*N-2), dtype=np.float64)
f_ext_dct1_1d[:N, :N] = f_grid
f_ext_dct1_1d[N:, :N] = f_grid[-2:0:-1, :]
f_ext_dct1_1d[:N, N:] = f_grid[:, -2:0:-1]
f_ext_dct1_1d[N:, N:] = f_grid[-2:0:-1, -2:0:-1]

f_ext_dct1_1d[0, :] *= np.sqrt(2)
f_ext_dct1_1d[:, 0] *= np.sqrt(2)
f_ext_dct1_1d[N-1, :] *= np.sqrt(2)
f_ext_dct1_1d[:, N-1] *= np.sqrt(2)

res_dct1_2d = (rfft2(f_ext_dct1_1d) * 0.5 * 1/(N-1))[:N, :N].real
res_dct1_2d[0, :] /= np.sqrt(2)
res_dct1_2d[:, 0] /= np.sqrt(2)
res_dct1_2d[N-1, :] /= np.sqrt(2)
res_dct1_2d[:, N-1] /= np.sqrt(2)

In [780]:
print("Error: ", np.sum(np.abs(f_base_dct1_2d-res_dct1_2d)))
f_base_dct1_2d, res_dct1_2d

Error:  8.10932742149721e-15


(array([[-2.60e+00,  4.44e-15,  1.02e+01, -5.65e-15],
        [ 4.61e-15, -8.88e-16, -3.26e-15,  1.26e-15],
        [ 1.02e+01, -1.78e-15, -1.16e+01,  3.77e-15],
        [-5.33e-15,  1.26e-15,  3.77e-15, -1.78e-15]]),
 array([[-2.60e+00,  4.61e-15,  1.02e+01, -5.33e-15],
        [ 3.77e-15, -8.88e-16, -2.37e-15,  1.26e-15],
        [ 1.02e+01, -3.26e-15, -1.16e+01,  3.77e-15],
        [-5.33e-15,  1.26e-15,  3.35e-15, -1.78e-15]]))

### 1.2. DCT Type II

In [593]:
f_base_dct2_1d = dct(f_base, 2, norm="ortho", orthogonalize=True)

f_ext_dct2_1d = np.zeros(4*N)
f_ext_dct2_1d[1:2*N:2] = f_base
f_ext_dct2_1d[2*N+1:4*N:2] = f_base[::-1]

res_dct2_1d = (rfft(f_ext_dct2_1d) * np.sqrt(2/N) * 0.5)[:N].real
res_dct2_1d[0] /= np.sqrt(2)

In [594]:
print("Error: ", np.sum(np.abs(f_base_dct2_1d-res_dct2_1d)))
f_base_dct2_1d, res_dct2_1d

Error:  3.497202527569243e-15


(array([12.7, -6.44, 0, -0.673, 0, -0.201, 0, -0.0507]),
 array([12.7, -6.44, 0, -0.673, 0, -0.201, 0, -0.0507]))

In [599]:
f_base_dct2_2d = dctn(f_grid, 2, norm="ortho", orthogonalize=True)

f_ext_dct2_2d = np.zeros((4*N, 4*N))
f_ext_dct2_2d[1:2*N:2, 1:2*N:2] = f_grid
f_ext_dct2_2d[2*N+1:4*N:2, 1:2*N:2] = f_grid[::-1, :]
f_ext_dct2_2d[1:2*N:2, 2*N+1:4*N:2] = f_grid[:, ::-1]
f_ext_dct2_2d[2*N+1:4*N:2, 2*N+1:4*N:2] = f_grid[::-1, ::-1]

res_dct2_2d = (rfft2(f_ext_dct2_2d) * (2/N) * 0.25)[:N, :N].real
res_dct2_2d[0, :] /= np.sqrt(2)
res_dct2_2d[:, 0] /= np.sqrt(2)

In [600]:
print("Error: ", np.sum(np.abs(f_base_dct2_2d-res_dct2_2d)))
f_base_dct2_2d, res_dct2_2d

Error:  6.676825531137018e-14


(array([[-2.63, 6.32e-15, 35.3, -1.27e-14, -24.5, 7.64e-15, 2.11, 4.16e-16],
        [6.65e-15, -5.4e-16, -7.36e-15, 2.09e-16, 3.38e-15, 9.64e-16, -6.08e-16, -9.18e-16],
        [35.3, -8.74e-15, -53.4, 1.88e-14, 23.2, -1.23e-14, -1.9, -1.12e-15],
        [-1.33e-14, 1.3e-16, 1.5e-14, 1.27e-15, -6.29e-15, -2.17e-15, 6.16e-16, 1.33e-16],
        [-24.5, 2.67e-15, 23.2, -5.46e-15, -6.02, 3.65e-15, 0.445, -5.3e-16],
        [7.57e-15, 1.04e-15, -8.64e-15, -2.28e-15, 3.42e-15, 9.51e-16, -1.24e-16, 1.78e-15],
        [2.11, -7.78e-16, -1.9, 1.47e-15, 0.445, -1.35e-15, -0.0319, 1.5e-15],
        [1.84e-15, -8.07e-16, -2.14e-15, 5.42e-17, 9.31e-17, 1.7e-15, 7.86e-16, -1.01e-15]]),
 array([[-2.63, 6.65e-15, 35.3, -1.33e-14, -24.5, 7.57e-15, 2.11, 1.84e-15],
        [6.53e-15, -6e-16, -8.29e-15, 1.18e-16, 2.67e-15, 1.14e-15, -1.37e-16, -7.3e-16],
        [35.3, -7.68e-15, -53.4, 1.52e-14, 23.2, -8.46e-15, -1.9, -2.21e-15],
        [-1.31e-14, 1.18e-16, 1.61e-14, 1.45e-15, -5.46e-15, -2.34e-15, 

### 1.3. DCT Type III

In [603]:
f_base_dct3_1d = dct(f_base, 3, norm="ortho", orthogonalize=True)

f_ext_dct3_1d = f_base * 2.0 * np.sqrt(N/2) * np.exp((np.arange(N) * np.pi * 0.5 / N)*1j)
f_ext_dct3_1d[0] *= np.sqrt(2)

res_dct3_1d = irfft(f_ext_dct3_1d, n=2*N)[:N]

In [604]:
print("Error: ", np.sum(np.abs(f_base_dct3_1d-res_dct3_1d)))
f_base_dct3_1d, res_dct3_1d

Error:  5.329070518200751e-15


(array([9.94, -8.8, 3.75, -2.95, 1.74, -1.26, 0.65, -0.244]),
 array([9.94, -8.8, 3.75, -2.95, 1.74, -1.26, 0.65, -0.244]))

In [605]:
f_base_dct3_2d = dctn(f_grid, 3, norm="ortho", orthogonalize=True)

def dct3_1d(inp):

    temp = inp * 2.0 * np.sqrt(N/2) * np.exp((np.arange(N) * np.pi * 0.5 / N)*1j)
    temp[0] *= np.sqrt(2)
    return irfft(temp, n=2*N)[:N].real

f_ext_dct3_2d = np.apply_along_axis(dct3_1d, 0, f_grid)
res_dct3_2d = np.apply_along_axis(dct3_1d, 1, f_ext_dct3_2d)

In [606]:
print("Error: ", np.sum(np.abs(f_base_dct3_2d-res_dct3_2d)))
f_base_dct3_2d, res_dct3_2d

Error:  1.5125227459389379e-13


(array([[-10.4, 13.5, 41.2, -4.59, -17.3, -2.22, 0.739, -0.23],
        [13.5, -10.2, -23.4, 2.25, 8.48, 1.03, -0.385, 0.0787],
        [41.2, -23.4, -39.3, 2.89, 10.9, 1.14, -0.56, 0.00424],
        [-4.59, 2.25, 2.89, -0.139, -0.528, -0.0354, 0.0341, 0.0104],
        [-17.3, 8.48, 10.9, -0.528, -2, -0.135, 0.129, 0.0389],
        [-2.22, 1.03, 1.14, -0.0354, -0.135, -0.00109, 0.0116, 0.00693],
        [0.739, -0.385, -0.56, 0.0341, 0.129, 0.0116, -0.0073, -0.000978],
        [-0.23, 0.0787, 0.00424, 0.0104, 0.0389, 0.00693, -0.000978, 0.00155]]),
 array([[-10.4, 13.5, 41.2, -4.59, -17.3, -2.22, 0.739, -0.23],
        [13.5, -10.2, -23.4, 2.25, 8.48, 1.03, -0.385, 0.0787],
        [41.2, -23.4, -39.3, 2.89, 10.9, 1.14, -0.56, 0.00424],
        [-4.59, 2.25, 2.89, -0.139, -0.528, -0.0354, 0.0341, 0.0104],
        [-17.3, 8.48, 10.9, -0.528, -2, -0.135, 0.129, 0.0389],
        [-2.22, 1.03, 1.14, -0.0354, -0.135, -0.00109, 0.0116, 0.00693],
        [0.739, -0.385, -0.56, 0.0341, 0.129, 

## 2. DSTs

### 2.1. DST Type I

In [777]:
f_base_dst1_1d = dst(f_base, 1, norm="ortho", orthogonalize=True)

f_ext_dst1_1d = np.zeros(2*N+2)
f_ext_dst1_1d[1:N+1] = f_base
f_ext_dst1_1d[N+2:] = -f_base[::-1]

res_dst1_1d = -0.5 * fft(f_ext_dst1_1d)[1:N+1].imag * np.sqrt(2 / (N+1))

In [778]:
print("Error: ", np.sum(np.abs(f_base_dst1_1d-res_dst1_1d)))
f_base_dst1_1d, res_dst1_1d

Error:  0.0


(array([ 4.87, -2.18,  1.15, -0.51]), array([ 4.87, -2.18,  1.15, -0.51]))

In [618]:
f_base_dst1_2d = dstn(f_grid, 1, norm="ortho", orthogonalize=True)

f_ext_dst1_2d = np.zeros((2*N+2, 2*N+2))
f_ext_dst1_2d[1:N+1, 1:N+1] = f_grid
f_ext_dst1_2d[N+2:, 1:N+1] = -f_grid[::-1, :]
f_ext_dst1_2d[1:N+1, N+2:] = -f_grid[:, ::-1]
f_ext_dst1_2d[N+2:, N+2:] = f_grid[::-1, ::-1]

res_dst1_2d = -0.5 * rfft2(f_ext_dst1_2d)[1:N+1, 1:N+1].real / (N+1)

In [619]:
print("Error: ", np.sum(np.abs(f_base_dst1_2d-res_dst1_2d)))
f_base_dst1_2d, res_dst1_2d

Error:  8.149772688176259e-14


(array([[-25.6, 1.11e-14, 49.7, -1.78e-14, -18.8, 7.11e-15, 2.21, -0],
        [1.11e-14, -8.44e-16, -6.42e-15, -5.63e-16, 2.58e-15, 1.28e-15, -4.06e-16, -3.87e-17],
        [49.7, -8.66e-15, -34.4, 1.33e-14, 9.52, -7.11e-15, -0.972, -0],
        [-1.59e-14, -4.93e-16, 1.14e-14, 1.67e-15, -4.25e-15, -7.69e-16, 4.07e-16, -1.94e-15],
        [-18.8, 5.27e-16, 9.52, -1.11e-15, -2.07, 1.78e-15, 0.18, -0],
        [9.01e-15, 1.37e-15, -5.4e-15, -1.37e-15, 1.3e-15, -3.94e-31, 4.96e-16, 1.37e-15],
        [2.21, 3.42e-16, -0.972, -7.22e-16, 0.18, 6.11e-16, -0.0133, -4.44e-16],
        [1.37e-15, -6.46e-16, -0, -2.4e-15, -1.37e-15, 4.1e-15, 1.37e-15, -2.27e-15]]),
 array([[-25.6, 1.11e-14, 49.7, -1.59e-14, -18.8, 9.01e-15, 2.21, 1.37e-15],
        [1.2e-14, -8.44e-16, -1.04e-14, -4.93e-16, 2e-15, 1.37e-15, 2.7e-16, -6.46e-16],
        [49.7, -6.42e-15, -34.4, 1.14e-14, 9.52, -5.4e-15, -0.972, -1.56e-31],
        [-1.49e-14, -5.63e-16, 1.23e-14, 1.67e-15, -2.35e-15, -1.37e-15, -6.45e-16, -2.4e-

### 2.2. DST Type II

In [607]:
f_base_dst2_1d = dst(f_base, 2, norm="ortho", orthogonalize=True)

f_ext_dst2_1d = f_base
f_ext_dst2_1d[1::2] = -f_base[1::2]

res_dst2_1d = dct(f_ext_dst2_1d, 2, norm="ortho", orthogonalize=True)[::-1]

In [608]:
print("Error: ", np.sum(np.abs(f_base_dst2_1d-res_dst2_1d)))
f_base_dst2_1d, res_dst2_1d

Error:  0.0


(array([11.5, -5.23, 4.05, -2.83, 2.71, -2.16, 2.29, -1.41]),
 array([11.5, -5.23, 4.05, -2.83, 2.71, -2.16, 2.29, -1.41]))

In [610]:
f_base_dst2_2d = dstn(f_grid, 2, norm="ortho", orthogonalize=True)

f_ext_dst2_2d = f_grid
f_ext_dst2_2d[1::2, ::2] = -f_grid[1::2, ::2]
f_ext_dst2_2d[::2, 1::2] = -f_grid[::2, 1::2]

res_dst2_2d = dctn(f_ext_dst2_2d, 2, norm="ortho", orthogonalize=True)[::-1, ::-1]

In [611]:
print("Error: ", np.sum(np.abs(f_base_dst2_2d-res_dst2_2d)))
f_base_dst2_2d, res_dst2_2d

Error:  0.0


(array([[-1.01e-15, 7.86e-16, 1.7e-15, 9.31e-17, 5.42e-17, -2.14e-15, -8.07e-16, 1.84e-15],
        [1.5e-15, -0.0319, -1.35e-15, 0.445, 1.47e-15, -1.9, -7.78e-16, 2.11],
        [1.78e-15, -1.24e-16, 9.51e-16, 3.42e-15, -2.28e-15, -8.64e-15, 1.04e-15, 7.57e-15],
        [-5.3e-16, 0.445, 3.65e-15, -6.02, -5.46e-15, 23.2, 2.67e-15, -24.5],
        [1.33e-16, 6.16e-16, -2.17e-15, -6.29e-15, 1.27e-15, 1.5e-14, 1.3e-16, -1.33e-14],
        [-1.12e-15, -1.9, -1.23e-14, 23.2, 1.88e-14, -53.4, -8.74e-15, 35.3],
        [-9.18e-16, -6.08e-16, 9.64e-16, 3.38e-15, 2.09e-16, -7.36e-15, -5.4e-16, 6.65e-15],
        [4.16e-16, 2.11, 7.64e-15, -24.5, -1.27e-14, 35.3, 6.32e-15, -2.63]]),
 array([[-1.01e-15, 7.86e-16, 1.7e-15, 9.31e-17, 5.42e-17, -2.14e-15, -8.07e-16, 1.84e-15],
        [1.5e-15, -0.0319, -1.35e-15, 0.445, 1.47e-15, -1.9, -7.78e-16, 2.11],
        [1.78e-15, -1.24e-16, 9.51e-16, 3.42e-15, -2.28e-15, -8.64e-15, 1.04e-15, 7.57e-15],
        [-5.3e-16, 0.445, 3.65e-15, -6.02, -5.46e-15,

### 2.3. DST Type III

In [612]:
f_base_dst3_1d = dst(f_base, 3, norm="ortho", orthogonalize=True)

f_ext_dst3_1d = f_base[::-1]
coeff = np.ones(N)
coeff[1::2] = -1

res_dst3_1d = dct(f_ext_dst3_1d, 3, norm="ortho", orthogonalize=True) * coeff

In [613]:
print("Error: ", np.sum(np.abs(f_base_dst3_1d-res_dst3_1d)))
f_base_dst3_1d, res_dst3_1d

Error:  0.0


(array([-0.955, 0.965, -0.989, 1.04, -1.14, 1.39, -2.31, 13.8]),
 array([-0.955, 0.965, -0.989, 1.04, -1.14, 1.39, -2.31, 13.8]))

In [614]:
f_base_dst3_2d = dstn(f_grid, 3, norm="ortho", orthogonalize=True)

f_ext_dst3_2d = f_grid[::-1, ::-1]
coeff = -np.ones((N, N))
coeff[1::2, 1::2] = 1
coeff[::2, ::2] = 1

res_dst3_2d = dctn(f_ext_dst3_2d, 3, norm="ortho", orthogonalize=True) * coeff

In [615]:
print("Error: ", np.sum(np.abs(f_base_dst3_2d-res_dst3_2d)))
f_base_dst3_2d, res_dst3_2d

Error:  0.0


(array([[-10.4, -13.5, 41.2, 4.59, -17.3, 2.22, 0.739, 0.23],
        [-13.5, -10.2, 23.4, 2.25, -8.48, 1.03, 0.385, 0.0787],
        [41.2, 23.4, -39.3, -2.89, 10.9, -1.14, -0.56, -0.00424],
        [4.59, 2.25, -2.89, -0.139, 0.528, -0.0354, -0.0341, 0.0104],
        [-17.3, -8.48, 10.9, 0.528, -2, 0.135, 0.129, -0.0389],
        [2.22, 1.03, -1.14, -0.0354, 0.135, -0.00109, -0.0116, 0.00693],
        [0.739, 0.385, -0.56, -0.0341, 0.129, -0.0116, -0.0073, 0.000978],
        [0.23, 0.0787, -0.00424, 0.0104, -0.0389, 0.00693, 0.000978, 0.00155]]),
 array([[-10.4, -13.5, 41.2, 4.59, -17.3, 2.22, 0.739, 0.23],
        [-13.5, -10.2, 23.4, 2.25, -8.48, 1.03, 0.385, 0.0787],
        [41.2, 23.4, -39.3, -2.89, 10.9, -1.14, -0.56, -0.00424],
        [4.59, 2.25, -2.89, -0.139, 0.528, -0.0354, -0.0341, 0.0104],
        [-17.3, -8.48, 10.9, 0.528, -2, 0.135, 0.129, -0.0389],
        [2.22, 1.03, -1.14, -0.0354, 0.135, -0.00109, -0.0116, 0.00693],
        [0.739, 0.385, -0.56, -0.0341, 0.129, 

# Tests

In [688]:
m = dct(np.eye(3), 1, norm="backward", orthogonalize=True, axis=0)
m

array([[1, 1.41, 1],
       [1.41, 0, -1.41],
       [1, -1.41, 1]])

In [646]:
m @ m

array([[4, 0, 4.44e-16],
       [0, 4, 0],
       [4.44e-16, 0, 4]])

In [736]:
dct(np.arange(4)+1, 1, norm="backward", orthogonalize=True)

array([12.07, -5.24,  2.07, -1.59])

In [653]:
m @ [0, 1, 2]

array([2, -1, 0])

In [658]:
f_ext_dct1_1d = np.concatenate((np.arange(3), np.arange(3)[-2:0:-1]))

(rfft(f_ext_dct1_1d)).real

array([4, -2, 0])

In [661]:
f_ext_dct1_1d

array([0, 1, 2, 1])

In [677]:
(rfft(np.eye(4), axis=0) @ f_ext_dct1_1d).real

array([4, -2, 0])

In [681]:
rm = rfft(np.eye(4), axis=0)
rm

array([[ 1.+0.j,  1.+0.j,  1.+0.j,  1.+0.j],
       [ 1.+0.j,  0.-1.j, -1.+0.j,  0.+1.j],
       [ 1.+0.j, -1.+0.j,  1.+0.j, -1.+0.j]])

In [698]:
res_rm = np.array([rm[:, 0], rm[:, 1] + rm[:, 3], rm[:, 2]]).T.real
res_rm

array([[1, 2, 1],
       [1, 0, -1],
       [1, -2, 1]])

In [702]:
res_rm @ np.diag([1, np.sqrt(2)/2, 1])

array([[1, 1.41, 1],
       [1, 0, -1],
       [1, -1.41, 1]])

In [705]:
(res_rm @ np.diag([1, np.sqrt(2)/2, 1])) @ np.arange(3)

array([3.41, -2, 0.586])

In [706]:
np.diag([1, np.sqrt(2)/2, 1]) @ np.arange(3)

array([0, 0.707, 2])

In [721]:
r = np.eye(3)
r[0, 1] = np.sqrt(2)/2
r

array([[1.  , 0.71, 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ]])

In [722]:
res_rm @ r 

array([[ 1.  ,  2.71,  1.  ],
       [ 1.  ,  0.71, -1.  ],
       [ 1.  , -1.29,  1.  ]])

In [719]:
np.set_printoptions(precision=2)

In [727]:
t = dct([0, 1, 2*np.sqrt(2)], 1, norm="backward", orthogonalize=False)
t[0] /= np.sqrt(2)
t[2] /= np.sqrt(2)
t

array([ 3.41, -2.83,  0.59])

In [735]:
f_ext_dct1_1d = np.concatenate((np.array([np.sqrt(2), 2, 3, 4*np.sqrt(2)]), np.array([np.sqrt(2), 2, 3, 4*np.sqrt(2)])[-2:0:-1]))

t = (rfft(f_ext_dct1_1d)).real
t[0] /= np.sqrt(2)
t[3] /= np.sqrt(2)
t

array([12.07, -5.24,  2.07, -1.59])

In [737]:
f_ext_dct1_1d

array([1.41, 2.  , 3.  , 5.66, 3.  , 2.  ])

In [784]:
rfft2(f_grid)

array([[-7.69e+01+0.00e+00j,  2.70e+02-1.11e-14j, -1.28e+02-0.00e+00j, -8.69e+00-1.11e-14j, -1.15e+01+0.00e+00j],
       [ 2.70e+02+0.00e+00j, -2.39e+02+4.00e-15j,  6.84e+01+0.00e+00j,  1.96e+00+9.65e-15j,  3.65e+00+0.00e+00j],
       [-1.28e+02+0.00e+00j,  6.84e+01-1.33e-15j, -8.24e+00+0.00e+00j,  8.82e-01-1.33e-15j,  5.94e-01+0.00e+00j],
       [-8.69e+00+0.00e+00j,  1.96e+00+4.00e-15j,  8.82e-01+0.00e+00j,  1.68e-01-1.66e-15j,  1.79e-01+0.00e+00j],
       [-1.15e+01+0.00e+00j,  3.65e+00+1.33e-15j,  5.94e-01+0.00e+00j,  1.79e-01+1.33e-15j,  1.81e-01+0.00e+00j],
       [-8.69e+00+0.00e+00j,  1.96e+00+4.00e-15j,  8.82e-01+0.00e+00j,  1.68e-01-1.66e-15j,  1.79e-01+0.00e+00j],
       [-1.28e+02+0.00e+00j,  6.84e+01-1.33e-15j, -8.24e+00+0.00e+00j,  8.82e-01-1.33e-15j,  5.94e-01+0.00e+00j],
       [ 2.70e+02+0.00e+00j, -2.39e+02+4.00e-15j,  6.84e+01+0.00e+00j,  1.96e+00+9.65e-15j,  3.65e+00+0.00e+00j]])